# Práctica 1: Reglas de asociación

Imports necesarios para la realización de la práctica

In [2]:
import numpy as np
import pandas as pd
import sklearn as sk
import mlxtend as mlx
import apyori as apy
import matplotlib as mpl
from mlxtend.preprocessing import TransactionEncoder

In [3]:
itemset = pd.read_csv("BlackFriday.csv").drop_duplicates().dropna()
itemset

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0
5,1000013,P00350442,M,46-50,1,C,3,1,2,3.0,15.0
6,1000013,P00155442,M,46-50,1,C,3,1,1,11.0,15.0
7,1000013,P0094542,M,46-50,1,C,3,1,2,4.0,9.0
8,1000015,P00161842,M,26-35,7,A,1,0,10,13.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...
233584,1006023,P00262242,M,26-35,0,C,2,1,1,11.0,16.0
233586,1006026,P00110742,M,36-45,6,C,1,1,1,2.0,8.0
233588,1006030,P00129842,M,26-35,17,C,1,1,6,8.0,15.0
233591,1006033,P00127642,M,51-55,13,B,1,1,1,2.0,15.0


Pregunta Nº1: Prueba al menos tres configuraciones de soporte y frecuencia para cada género, edad y tipo de producto. (1 punto)


In [4]:
te = TransactionEncoder()
te_ary = te.fit(itemset).transform(itemset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df
apy.apriori(df, min_support=0.01, use_colnames=True, verbose=1)

<generator object apriori at 0x000001AAB054CDD0>

Pregunta Nº2: ¿Qué diferencias hay entre usar soporte y frecuencia? Respalda la respuesta con datos (1 punto)

Pregunta Nº3: ¿Qué tipo reglas desaparecen según la configuración y categorías (género, edad y tipo de producto) consideradas? ¿Por qué? (1 punto)

Pregunta Nº4: Para una de las configuraciones, interpreta algunas de las reglas que te hayan resultado interesantes obtenidas usando la clase association_rules y un par de configuraciones cambiando la variable metric y min_threshold. Justifica los resultados. (1 punto)


Pregunta Nº5: Dadas las mejores configuraciones. ¿Existen reglas o patrones que se repitan? ¿Podemos generalizar de alguna manera como se comportan los clientes? Expón las conclusiones respaldadas con datos (1 punto)